In [ ]:
import matplotlib.pyplot as plt

import pandas as pd
import geopandas
from geodatasets import get_path

from dwca.read import DwCAReader

# Wolf data set

In [ ]:
useful_columns = ["id", "eventDate", "decimalLatitude", "decimalLongitude", "coordinateUncertaintyInMeters", 
                  "scientificName", "level0Gid", "level0Name", "level1Gid", "level1Name", "level2Gid", "level2Name", 
                  "level3Gid", "level3Name", "geodeticDatum"]

In [ ]:
with DwCAReader("./datasets/0054964-231002084531237.zip") as dwca:
    df = dwca.pd_read('occurrence.txt', parse_dates=True)
    df = df[useful_columns]
    assert (df["geodeticDatum"][0] == df["geodeticDatum"]).all(), "Ovservations do not share coordinate reference system!"
    gdf = geopandas.GeoDataFrame(df, 
                                 geometry=geopandas.points_from_xy(df["decimalLongitude"], df["decimalLatitude"]), 
                                 crs=df["geodeticDatum"][0])

In [ ]:
world = geopandas.read_file(get_path("naturalearth.land"))

# We restrict to South America.
ax = world.clip([-11, 45, 24, 60]).plot(color="white", edgecolor="black")

# We can now plot our ``GeoDataFrame``.
gdf.plot(ax=ax, color="red")

plt.show()